**Goal:** represent the tracks downloaded from [Wikiloc](http://wikiloc.com/) or transformed into .gpx format from Garmin .fit files, using for example the [gpsbabel](http://www.gpsbabel.org/) tool.

In [1]:
import re
import os
import folium
import glob
import logging
import numpy as np
from geopy.geocoders import Nominatim
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import colors

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [16]:
print folium.__version__

0.2.1


# User inputs

Define the list of directories where the files are stored and where the html will be written.

In [47]:
region = 'hiking'
gpxdirlist = ['/home/ctroupin/DataOceano/GPX/Mallorca/' + s for s in ['hike', 'run', 'trail']] 
outputdir = '/home/ctroupin/public_html/LeafletMaps'
mapname = region + '.html'
gpxbasename = '*.gpx'

Generate a lists (of lists) of files.

In [48]:
gpxfilelists = [sorted(glob.glob(os.path.join(gpxdir, gpxbasename))) for gpxdir in gpxdirlist]

## Logging

In [19]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='/home/ctroupin/logs/GPXheadmap.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

# Initialise the map

In [20]:
geolocator = Nominatim()
#location = geolocator.geocode("Lloret de Vistalegre, Mallorca, Spain")
#location = geolocator.geocode("Verviers, Belgique")
#location = geolocator.geocode("Gran Canaria, Spain")
location = geolocator.geocode("La Palma, Spain")
logger.info("Working on location: %s" %(location[0]))

# Functions to read GPX

In [21]:
def read_gpx(filename):
    coords = []
    with open(filename) as f:
        for lines in f:
            match = re.search('<trkpt lat="([-0-9\.]+)" lon="([-0-9\.]+)">', lines)
            if match:
                coords.append((float(match.group(1)), float(match.group(2))))
    return coords

In [22]:
def get_trackname(filename):
    '''
    Get the track name from a GPX file
    '''
    with open(filename) as f:
        match = None
        while match == None:
            line = f.readline()
            match = re.search('<name>(.+)</name>', line)
            if match:
                trackname = match.group(1)
    return trackname

In [23]:
def create_gradient(cb, ncolors):
    '''
    From a given matplotlib color palette, 
    provide a gradient dictionnary to be passed to 
    the heatmap function of folium
    INPUTS: cb = matplotlib colormap
            ncolors = number of colors
    OUTPUT: gradient = dictionnary containing the color
    '''
    gradient = {}
    for i, j in zip(np.linspace(0.35, 1.0, ncolors), np.linspace(0., cb.N-1, ncolors).astype('uint8')):
        gradient[i] = colors.rgb2hex(cb.colors[j])
    return gradient

# Create the map

In [49]:
logger.info('Creating a new map')
logger.info('Preparing basemaps')

## Set basemaps and center

In [50]:
map_run = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=10, tiles='Stamen Terrain')
map_run.add_tile_layer(tiles=u'OpenStreetMap', active=False)
map_run.add_tile_layer(tiles=u'cartodbdark_matter', active=False)

In [51]:
logger.info('Create gradient dictionnary')
gradient = create_gradient(plt.cm.plasma, 10)

## Loop on the files

In [52]:
logger.info('Create gradient dictionnary')
gradient = create_gradient(plt.cm.plasma, 10)
coordsall = []
logger.info('Loop on the GPX files')

for gpxfilelist in gpxfilelists:
    logger.info('Working on directory {0}'.format(gpxfilelist))
    for gpxfiles in gpxfilelist:
        logger.debug(gpxfiles)
        #trackname = get_trackname(gpxfiles)
        #logger.debug(trackname)
        coords = read_gpx(gpxfiles)
        coordsall.append(coords)
        folium.PolyLine(coords, color="yellow", weight=2).add_to(map_run)
        #icon = folium.features.CustomIcon(iconurl, icon_size=(50, 50))
        #folium.Marker(location=coords[0], popup=gpxfiles).add_to(map_run)
    
coords2plot = [item for coords in coordsall for item in coords] 
folium.plugins.HeatMap(coords2plot, radius=2, blur=1, name="Heatmap",
                       gradient=gradient).add_to(map_run)
folium.LayerControl().add_to(map_run)
# map_run

## Save the map

In [53]:
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    logging.info('Creating directory %s' %(outputdir))
map_run.save(os.path.join(outputdir, mapname))
outputdir

'/home/ctroupin/public_html/LeafletMaps'